In [ ]:
# issue: HTTP ERROR 429 after opening too many links (Too Many Requests)

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from bs4 import BeautifulSoup

driver = webdriver.Chrome()
driver.maximize_window()
driver.get("https://www.instagram.com")

In [ ]:
# log in first account
username = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.NAME, "username"))
)
password = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.NAME, "password"))
)
login = driver.find_element(By.XPATH, '//*[@id="loginForm"]/div/div[3]/button/div')

ID = "your_username"
username.clear()
password.clear()
username.send_keys(ID)
password.send_keys("your_password")

login.click()
# wait until search box appears
wait = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="react-root"]/section/nav/div[2]/div/div/div[2]/input'))
)

In [ ]:
# crawl saved collection from first account
driver.get("https://www.instagram.com/" + ID + "/saved/all-posts/")
time.sleep(3)

links = []

last_height = driver.execute_script("return document.body.scrollHeight")
last_link = ""

while True:
    # add links
    soup = BeautifulSoup(driver.page_source, "html.parser")
    articles = soup.find("article")
    posts = articles.find_all("div", "_aabd _aa8k _aanf")
    start_appending = False

    for post in posts:
        info = post.find("a", "qi72231t nu7423ey n3hqoq4p r86q59rh b3qcqh3k fq87ekyn bdao358l fsf7x5fv rse6dlih s5oniofx m8h3af8h l7ghb35v kjdc1dyq kmwttqpk srn514ro oxkhqvkx rl78xhln nch0832m cr00lzj9 rn8ck1ys s3jn8y49 icdlwmnq _a6hd")
        link = info.attrs['href']
        
        if last_link == "":
            start_appending = True

        if start_appending:
            links.append("https://www.instagram.com" + link)
            last_link = link
        elif link == last_link:
            start_appending = True

    # scroll and load
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(7)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

In [ ]:
# log out first account from saved collection page
icon = driver.find_element(By.XPATH, "/html/body/div[1]/div/div/div/div[1]/div/div/div/div[1]/section/nav/div[2]/div/div/div[3]/div/div[6]/div[1]")
icon.click()

logout = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div/div/div/div[1]/div/div/div/div[1]/section/nav/div[2]/div/div/div[3]/div/div[6]/div[2]/div[2]/div[2]/div[5]"))
)

logout.click()
time.sleep(3)

In [ ]:
# log in second acccount
username = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.NAME, "username"))
)
password = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.NAME, "password"))
)
login = driver.find_element(By.XPATH, '//*[@id="loginForm"]/div/div[3]/button/div')

ID2 = "your_username2"
username.clear()
password.clear()
username.send_keys(ID2)
password.send_keys("your_password2")

login.click()
# wait until search box appears
wait = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="react-root"]/section/nav/div[2]/div/div/div[2]/input'))
)

In [ ]:
# transfer saved posts to second account
success = 0
fail = 0

for link in reversed(links):
    driver.get(link)

    try:
        save = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div/div/div/div[1]/div/div/div/div[1]/section/main/div[1]/div[1]/article/div/div[2]/div/div[2]/section[1]/span[4]/div/div/button"))
        )
        save.click()
        success = success + 1
    except:
        fail = fail + 1
        continue

print("Total saved-posts: " + str(len(links)))
print("Sucessfully transferred: " + str(success))
print("Failed: " + str(fail))